In [1]:
from __future__ import print_function, division
import os, random, time, copy
import numpy as np
import pandas as pd
import os.path as path
import scipy.io as sio
from scipy import misc
from scipy import ndimage, signal
import scipy
import pickle
import sys
import math
import matplotlib.pyplot as plt
from io import BytesIO


import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler 
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
import torchvision.utils as vutils


import warnings 
warnings.filterwarnings("ignore")
print(sys.version)
print(torch.__version__)


manualSeed = 999
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

3.6.13 |Anaconda, Inc.| (default, Feb 23 2021, 21:15:04) 
[GCC 7.3.0]
1.8.0+cu111
Random Seed:  999


In [2]:
torch.manual_seed(0)

device ='cpu'
if torch.cuda.is_available(): 
    device='cuda:0'


batch_size = 16    
bestEpoch = 4  
lr = 0.0001 
num_epochs = 5
torch.cuda.device_count()
torch.cuda.empty_cache()

save_dir="./savedata/RPGAN"

print(save_dir)    
if not os.path.exists(save_dir): os.makedirs(save_dir)

log_filename = os.path.join(save_dir, 'train.log')

./savedata/RPGAN


In [3]:

print('==> Preparing data.............................')


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.datasets import MNIST, CIFAR10, CIFAR100, SVHN

import os
import torch
from torch import nn,optim
import torch.nn.functional as F
from torchvision import datasets, transforms

from time import perf_counter

import  numpy as np
import torch.utils.data as Data

from torch.utils.data import Dataset, DataLoader

class Safeman(Dataset):
    
    def __init__(self, data,targets):
        super(Safeman, self).__init__()
        self.data = data
        self.targets = targets
        
     
    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        img, target = self.data[idx], self.targets[idx]
        return img, target

class Safeman_Filter(Safeman):

    def __Filter__(self, known):
        targets = self.targets.data.numpy()
        mask, new_targets = [], []
        for i in range(len(targets)):
            if targets[i] in known:
                mask.append(i)
                new_targets.append(known.index(targets[i]))
        self.targets = np.array(new_targets)
        mask = torch.tensor(mask).long()
        self.data = torch.index_select(self.data, 0, mask)
        
class Safeman_FilterB(Safeman):

    def __Filter__(self, known):
        targets = self.targets.data.numpy()
        new_targets = []
        for i in range(len(targets)):
            if targets[i] in known:
                new_targets.append(0)
            else:
                new_targets.append(1)
        self.targets = np.array(new_targets)
        self.data = self.data

class Safeman_FilterC(Safeman):
    
    def __Filter__(self, trainknown):
        train_class_num=len(trainknown)
        for i in range(0,len(self.targets)) :
            if self.targets[i]>train_class_num:
                self.targets[i] = train_class_num
        self.data = self.data

        
class Safeman_FilterD(Safeman):

    def __Filter__(self, known):
        targets = self.targets.data.numpy()
        mask, new_targets = [], []
        train_class_num=len(known)
        for i in range(len(targets)):
            if targets[i] not in known:
                mask.append(i)
                new_targets.append(train_class_num)
        self.targets = np.array(new_targets)
        mask = torch.tensor(mask).long()
        self.data = torch.index_select(self.data, 0, mask)


        
def setup_seed(seed):

    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True

setup_seed(8)



known=[0, 1, 2,3,4]
unknown=[ 5,6,7]

num_class=len(known)



X_train0 = np.load('./dataset/X_train_ukm.npy')
y_train1 = np.load('./dataset/y_train_ukm.npy')
X_final_test0 = np.load('./dataset/X_final_test_ukm.npy' )
y__final_test1 = np.load('./dataset/y__final_test_ukm.npy')

X_train1=[]
X_final_test1=[]

for i in range(len(y_train1)):
    a = np.resize(X_train0[i], (3, 32, 32))
    X_train1 += [a]
    
for j in range(len(y__final_test1)):
    b = np.resize(X_final_test0[j], (3, 32, 32))
    X_final_test1 += [b]

i=0
j=0



x_train, x_test, y_train,y_test = torch.Tensor(X_train1), torch.Tensor(X_final_test1), torch.from_numpy(y_train1), torch.from_numpy(y__final_test1)

print(x_train.shape, x_test.shape, y_train.shape,y_test.shape)

train_dataset = Data.TensorDataset(x_train, y_train)
train_dataset.data = train_dataset.tensors[0]
train_dataset.targets = train_dataset.tensors[1]



test_dataset = Data.TensorDataset(x_test, y_test)
test_dataset.data = test_dataset.tensors[0]
test_dataset.targets = test_dataset.tensors[1]



labels =['ARP poisining', 'BeEF HTTP exploits','Mass HTTP requests','Metasploit exploits','Normal','Port scanning','TCP flood','UDP data flood']



train_dataset.classes = labels
test_dataset.classes = labels

train_dataset.classes_to_idx = {i: label for i, label in enumerate(labels)}
test_dataset.classes_to_idx = {i: label for i, label in enumerate(labels)}



b_s=16

trainset = Safeman_Filter(data=train_dataset.data,targets=train_dataset.targets)
print('All down Train Data:', len(trainset))
trainset.__Filter__(known=known)


train_loader = torch.utils.data.DataLoader(
    trainset, batch_size=b_s, shuffle=True,
    num_workers=4,drop_last=True)


print('Real train Data:', len(trainset))



testsetA = Safeman_Filter(data=test_dataset.data,targets=test_dataset.targets)
print('All testsetA Data:', len(testsetA))
testsetA.__Filter__(known=known)


test_loader_A = torch.utils.data.DataLoader(
    testsetA, batch_size=b_s, shuffle=False,
    num_workers=4,drop_last=True)


print('Real testsetA Data:', len(testsetA))


testsetD = Safeman_FilterD(data=test_dataset.data,targets=test_dataset.targets)
print('All testsetA Data:', len(testsetA))
testsetD.__Filter__(known=known)


test_loader_D = torch.utils.data.DataLoader(
    testsetD, batch_size=b_s, shuffle=False,
    num_workers=4,drop_last=True)


print('Real testsetD Data:', len(testsetD))


testsetC = Safeman_FilterC(data=test_dataset.data,targets=test_dataset.targets)
print('All testsetC Data:', len(testsetC))
testsetC.__Filter__(trainknown=known)


test_loader_C = torch.utils.data.DataLoader(
    testsetC, batch_size=b_s, shuffle=False,
    num_workers=4,drop_last=True)


print('Real testsetC Data:', len(testsetC))


print("done!")

==> Preparing data.............................
torch.Size([10309, 3, 32, 32]) torch.Size([2578, 3, 32, 32]) torch.Size([10309]) torch.Size([2578])
All down Train Data: 10309
Real train Data: 8923
All testsetA Data: 2578
Real testsetA Data: 2226
All testsetA Data: 2226
Real testsetD Data: 352
All testsetC Data: 2578
Real testsetC Data: 2578
done!


In [4]:
      
class Generatorzy(nn.Module):
    def __init__(self, z_dim):
        super(Generatorzy, self).__init__()
        
        self.fc = nn.Linear(z_dim, 256*8*8)
        self.g_deconv_1 = nn.Sequential(
                          nn.ConvTranspose2d(256, 128, kernel_size=3,
                                    stride= 2, padding=(3-2+1)//2,
                                    output_padding = (3-2)%2), 
                          nn.BatchNorm2d(128),
                          nn.LeakyReLU()
                          )
        self.g_deconv_2 = nn.Sequential(
                          nn.ConvTranspose2d(128, 64, kernel_size=3,
                                    stride= 1, padding=(3-1+1)//2,
                                    output_padding = (3-1)%2), 
                          nn.BatchNorm2d(64),
                          nn.LeakyReLU()
                          )
        self.g_deconv_3 = nn.Sequential(
                          nn.ConvTranspose2d(64, 3, kernel_size=3,
                                    stride= 2, padding=(3-2+1)//2,
                                    output_padding = (3-2)%2),
                          nn.Tanh()
                          )
        
        

    def forward(self, x):
        x = self.fc(x).view(-1, 256, 8, 8)
        x = self.g_deconv_1(x)
        x = self.g_deconv_2(x)
        x = self.g_deconv_3(x)
        
        return x     


    
class Discriminatorzy(nn.Module):
    def __init__(self):
        super(Discriminatorzy, self).__init__()
        
        self.d_conv_1 = nn.Sequential(
                          nn.Conv2d(3, 32, kernel_size=3,
                                    stride=2, padding=1), 
                          nn.LeakyReLU()
                          )
        self.d_conv_2 = nn.Sequential(
                          nn.Conv2d(32, 64, kernel_size=3,
                                    stride=2, padding=1), 
                          nn.LeakyReLU()
                          )
        self.d_conv_3 = nn.Sequential(
                          nn.Conv2d(64, 128, kernel_size=3,
                                    stride=2, padding=0), 
                          nn.LeakyReLU()
                          )
        self.fc = nn.Linear(3*3*128, 1)
        self.fczy = nn.Linear(3*3*128, 64)

    def forward(self, x):
        x = self.d_conv_1(x)
        x = self.d_conv_2(x)
        x = self.d_conv_3(x)
        x = x.view(-1, 128*3*3)
        x_zy = self.fczy(x)
        x = torch.sigmoid(self.fc(x))
        return x_zy,x

In [5]:
print(device)

bestEpoch =2 
z_dim=100

discriminator = Discriminatorzy()
netDzy = discriminator.to(device)
generator = Generatorzy(z_dim)
netGzy = generator.to(device)


path_to_G = os.path.join(save_dir, 'RPGAN-epoch-{}.GNet'.format(bestEpoch))
path_to_D = os.path.join(save_dir, 'RPGAN-epoch-{}.DNet'.format(bestEpoch))
netGzy.load_state_dict(torch.load(path_to_G))
netDzy.load_state_dict(torch.load(path_to_D))


cuda:0


<All keys matched successfully>

In [6]:

featesttrain=[]

for sample in train_loader:    
    datazy, label = sample
    datazy = datazy.to(device)
    label = label.type(torch.long).view(-1).to(device)    
    features1,_ = netDzy(datazy) 
    featesttrain+=features1

torch.save(featesttrain,'./savedata/featesttrain0102.pkl')

In [7]:
featestA=[]

dataloader_test_closeset= test_loader_A

for sample in dataloader_test_closeset:    
    datazy, label = sample
    datazy = datazy.to(device)
    label = label.type(torch.long).view(-1).to(device)    
    features1,_ = netDzy(datazy)
    featestA+=features1


torch.save(featestA,'./savedata/featestA0102.pkl')

In [8]:
featestD=[]

dataloader_openset = test_loader_D

for sample in dataloader_openset:
    datazy, label = sample
    datazy = datazy.to(device)    
    features1,_ = netDzy(datazy) 
    featestD+=features1


torch.save(featestD,'./savedata/featestD0102.pkl')


print("save done!")

save done!
